In [ ]:
from google.colab import drive
import os
import itertools
import shutil
import random
import glob
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import files
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, Dropout, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model

In [ ]:
%cd /content/drive/MyDrive/Dataset/

/content/drive/MyDrive/Dataset


In [ ]:
! unzip covid19-radiography-database

In [ ]:
if os.path.isdir('data/train/covid') is False:
  os.makedirs('data/train/normal')
  os.makedirs('data/train/covid')
  os.makedirs('data/test/normal')
  os.makedirs('data/test/covid')
  os.makedirs('data/valid/normal')
  os.makedirs('data/valid/covid')

In [ ]:
os.chdir('/content/drive/My Drive/Dataset/COVID-19_Radiography_Dataset/COVID')

In [ ]:
for c in random.sample(glob.glob('COVID-*'), 2531):
  shutil.move(c, '../../data/train/covid')

for c in random.sample(glob.glob('COVID-*'), 723):
  shutil.move(c, '../../data/valid/covid')

for c in random.sample(glob.glob('COVID-*'), 362):
  shutil.move(c, '../../data/test/covid')

In [ ]:
os.chdir('/content/drive/My Drive/Dataset/COVID-19_Radiography_Dataset/Normal')

In [ ]:
for c in random.sample(glob.glob('Normal-*'), 2531):
  shutil.move(c, '../../data/train/normal')

for c in random.sample(glob.glob('Normal-*'), 723):
  shutil.move(c, '../../data/valid/normal')

for c in random.sample(glob.glob('Normal-*'), 362):
  shutil.move(c, '../../data/test/normal')

In [ ]:
os.chdir('/content/drive/My Drive/Dataset/')

In [ ]:
train_path = '/content/drive/My Drive/Dataset/data/train'
valid_path = '/content/drive/My Drive/Dataset/data/valid'
test_path =  '/content/drive/My Drive/Dataset/data/test'

In [ ]:
train_gen = ImageDataGenerator(rescale=1.0/255.0).flow_from_directory(directory=train_path, target_size=(299, 299), classes=['covid','normal'])
valid_gen = ImageDataGenerator(rescale=1.0/255.0).flow_from_directory(directory=valid_path, target_size=(299, 299), classes=['covid','normal'])
test_gen = ImageDataGenerator(rescale=1.0/255.0).flow_from_directory(directory=test_path, target_size=(299, 299), classes=['covid','normal'], shuffle=False)

Found 5062 images belonging to 2 classes.
Found 1446 images belonging to 2 classes.
Found 724 images belonging to 2 classes.


In [ ]:
model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(3,3), input_shape=(299,299,3), activation='relu'))
model.add(Conv2D(filters=128, kernel_size=(3,3), activation='relu'))
model.add(MaxPool2D(pool_size=(2,2),strides=2))
model.add(Dropout(0.25))

model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu'))
model.add(MaxPool2D(pool_size=(2,2),strides=2))
model.add(Dropout(0.25))

model.add(Conv2D(filters=128, kernel_size=(3,3), activation='relu'))
model.add(MaxPool2D(pool_size=(2,2),strides=2))
model.add(Dropout(0.25))

model.add(Conv2D(filters=512, kernel_size=(3,3), activation='relu'))
model.add(MaxPool2D(pool_size=(2,2),strides=2))
model.add(Dropout(0.25))

model.add(Conv2D(filters=512, kernel_size=(3,3), activation='relu'))
model.add(MaxPool2D(pool_size=(2,2),strides=2))
model.add(Dropout(0.25))

model.add(Flatten())


model.add(Dense(64, activation='relu'))
model.add(Dropout(0.25))

model.add(Dense(2, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam',
             metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 297, 297, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 295, 295, 128)     36992     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 147, 147, 128)    0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 147, 147, 128)     0         
                                                                 
 conv2d_2 (Conv2D)           (None, 145, 145, 64)      73792     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 72, 72, 64)       0         
 2D)                                                    

In [ ]:
model.fit(
    x=train_gen,
    validation_data=valid_gen,
    epochs=50)

Epoch 1/50
159/159 [==============================] - 1291s 8s/step - loss: 0.6871 - accuracy: 0.5603 - val_loss: 0.6320 - val_accuracy: 0.6971
Epoch 2/50
159/159 [==============================] - 124s 781ms/step - loss: 0.5734 - accuracy: 0.7049 - val_loss: 0.4983 - val_accuracy: 0.7898
Epoch 3/50
159/159 [==============================] - 124s 781ms/step - loss: 0.4212 - accuracy: 0.8242 - val_loss: 0.3182 - val_accuracy: 0.8700
Epoch 4/50
159/159 [==============================] - 124s 781ms/step - loss: 0.3474 - accuracy: 0.8609 - val_loss: 0.2627 - val_accuracy: 0.9025
Epoch 5/50
159/159 [==============================] - 124s 781ms/step - loss: 0.2733 - accuracy: 0.8888 - val_loss: 0.2251 - val_accuracy: 0.9225
Epoch 6/50
159/159 [==============================] - 124s 780ms/step - loss: 0.2461 - accuracy: 0.9038 - val_loss: 0.2234 - val_accuracy: 0.9122
Epoch 7/50
159/159 [==============================] - 124s 780ms/step - loss: 0.2021 - accuracy: 0.9226 - val_loss: 0.1920 - v

In [ ]:
metrics = pd.DataFrame(model.history.history)

In [ ]:
if os.path.isdir('data/dataframes') is False:
  os.makedirs('data/dataframes')

if os.path.isdir('data/models') is False:
  os.makedirs('data/models')

In [ ]:
metrics.to_csv('data/dataframes/metrics.csv', index=False)
model.save('data/models/cnn_model.h5')

In [ ]:
files.download('data/dataframes/metrics.csv')
files.download('data/models/cnn_model.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>